## Imports

In [1]:
import os
import sys
import json
import gc
import re
from tqdm import notebook as tqdm
import pickle
import itertools
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from transformers import AutoTokenizer, AutoModel

import torch

SEP_TOKEN = '<sep>'

/home/eduard/dep-tasks/prob-topic-model/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sns.set(font_scale=1.2, palette='Set2')

plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 12
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 24
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 24
plt.rcParams["figure.figsize"] = (12, 7)

SEED = 123

## Text preprocessing

Смотрим на распределение секций внутри текста.

Секция -- всё, что обёрнуто в `==`

Из примеров выше: `== Майндмап ==`, `== Аннотация ==`, `== Видео ==`

Далее фильтруем по числу документов в секции: их должно быть не менее 20.

В конце сортируем по числу документов в секции.

In [3]:
with open('0x1tv-dataset.pickle', 'rb') as fd:
    data = pickle.load(fd)

Оставляем 3 домена: заголовок, аннотация и тезисы

In [4]:
def get_min_position(*positions):
    return min([pos for pos in positions if pos >= 0], default=len(data))

def extract_section_text(data, section_name):
    if section_name == 'Аннотация':
        blockquote_pattern = r'<blockquote>(.*?)</blockquote>'
        match = re.search(blockquote_pattern, data, re.DOTALL)
        return match.group(1).strip() if match else None
    
    section_pattern = fr'==\s*{re.escape(section_name)}\s*=='
    section_start = re.search(section_pattern, data)
    
    if not section_start:
        return None
   
    position = section_start.end()

    # Look for stop tokens and the start of next sections
    stop_tokens = ['{{----}}', '{{LinksSection}}', '== Примечания и отзывы ==']
    ends = [data.find(token, position) for token in stop_tokens]
    end_position = get_min_position(*ends)
    
    if end_position != len(data):
        return data[position:end_position].strip()
    
    return data[position:].strip()

def extract_sections(data):
    sections = ['Аннотация', 'Thesis', 'Тезисы', 'Расширенные тезисы']
    extracted_texts = {section: extract_section_text(data, section) for section in sections}
    return extracted_texts


def coalesce(dct, keys):
    for key in keys:
        if dct[key] is not None or len(dct[key]) > 0:
            return dct[key]
    return None


def parse_document(doc):
    parsed_doc = dict()
    parsed_doc['title'] = doc['title']
    
    sections_data = extract_sections(doc['text'])
    parsed_doc['annotation'] = sections_data['Аннотация']
    parsed_doc['thesis'] = coalesce(sections_data, ['Thesis', 'Тезисы', 'Расширенные тезисы'])
    
    return parsed_doc

Парсим ФИО докладчиков

In [5]:
def speaker_detect(text):
    speakers = re.findall('(?<={{Speaker\|).*(?=}})', text)
    preproc_speakers = []
    for speaker in speakers:
        preproc_speakers.extend(speaker.split('|'))
    return preproc_speakers


all_speakers = [speaker_detect(text['text']) for text in data['articles']]

print('Число докладов без спикеров:', len(list(filter(lambda x: len(x) == 0, all_speakers))))
print('Число докладов с одним спикером:', len(list(filter(lambda x: len(x) == 1, all_speakers))))
print('Число докладов с двумя спикерами:', len(list(filter(lambda x: len(x) == 2, all_speakers))))
print('Число докладов с не менее тремя спикерами:', len(list(filter(lambda x: len(x) >= 3, all_speakers))))

Число докладов без спикеров: 74
Число докладов с одним спикером: 2313
Число докладов с двумя спикерами: 119
Число докладов с не менее тремя спикерами: 35


Фильтруем категории

In [6]:
def clear_categories(sample):
    return [re.sub('Категория:', '', el) for el in sample]


text_categories = [clear_categories(item['categories']) for item in data['articles']]
child_categories = [item['title'] for item in data['categories']]
parent_categories = [clear_categories(item['categories']) for item in data['categories']]

ALL_CATEGORIES = list(set(itertools.chain(*text_categories)) | set(child_categories) | set(itertools.chain(*parent_categories)))

child_to_parent_categories = {item['title']: clear_categories(item['categories']) for item in data['categories']}

Удалим вручную все имена из списка категорий + `HasSpeaker`

In [7]:
ALL_CATEGORIES_WITHOUT_NAMES = set([
    '.NET',
    '1C',
    'ALT Linux',
    'ALTLinux на Эльбрусе',
    'AR',
    'AWS',
    'Accessibility',
    'Agile',
    'Agile Introduction',
    'Agile process',
    'Agile в корпорациях',
    'Agile — технологические практики',
    'Agile&Lean Mindset',
    'Agile-культура',
    'Agile-масштабирование',
    'Agile-преобразования',
    'Alfresco',
    'Ansible',
    'Arduino',
    'AstraLinux',
    'Atlassian',
    'Azure',
    'B2B продукты',
    'BDD',
    'BigData',
    'Blockchain',
    'Bluemix',
    'Business rules engine',
    'C++',
    'CMMI',
    'CQRS',
    'CRIU',
    'Clouds',
    'Clsync',
    'Code Review',
    'Collaboration tools',
    'Configuration Management',
    'ContactOK',
    'Continuous Integration',
    'CouchDB',
    'Csharp',
    'CustisWikiToLib',
    'Customer Journey Map',
    'DDD',
    'DSL-языки',
    'Data Analysis',
    'Deployment',
    'Design Thinking',
    'DevOps',
    'Draft',
    'E-commerce',
    'Embox',
    'Erlang',
    'Extreme Programming',
    'Feature Branches',
    'Firefox',
    'Foresight management',
    'FreeIPA',
    'Front end development',
    'Fsharp',
    'GWT',
    'Glibc',
    'Go',
    'Groovy',
    'Growth Hacking',
    'HR',
    'Hardware',
    'Health',
    'High Performace Computing',
    'Highload-архитектуры',
    'IP-телефония',
    'IT-законы',
    'IT-образование',
    'Impact Map',
    'Information Security',
    'Internet of Thing',
    'Java',
    'Java EE',
    'Javascript',
    'Jenkins',
    'Kanban',
    'Knowledge Management',
    'Kotlin',
    'Kubernetes',
    'LAMP',
    'LSM',
    'LeSS',
    'Lean',
    'Lean Startup',
    'Legacy',
    'Libreoffice',
    'Linux',
    'Linux для Эльбруса',
    'Linux-дистрибутивы',
    'Linux-дистрибутивы для Enterprise',
    'Linux-обучение',
    'Lua',
    'MIPS',
    'Machine Learning',
    'Mechanics',
    'Microsoft',
    'Misc',
    'Mkimage-profiles',
    'MongoDB',
    'Morpheus',
    'MySQL',
    'NLP',
    'NOSQL',
    'NUI',
    'Natural Language Processing',
    'NeedContacts',
    'Nemerle',
    'Node.js',
    'Object Oriented Programming',
    'Open-source',
    'Open-source CAD',
    'Open-source CMS',
    'Open-source CRM',
    'Open-source ERP',
    'Open-source PAAS',
    'Open-source TCO',
    'Open-source and Community',
    'Open-source and hardware',
    'Open-source communications',
    'Open-source for Enterprise',
    'Open-source operating systems',
    'Open-source projects',
    'Open-source СУБД',
    'Open-source и законы',
    'OpenShift',
    'OpenStack',
    'OpenVZ',
    'PAAS',
    'PHP',
    'Pascale Xelot-Dugat',
    'People Management',
    'Pivot',
    'PostgreSQL',
    'ProductMeetup',
    'Python',
    'RISC-V',
    'ROSA Linux',
    'ROSALab',
    'RTOS',
    'Reviewed',
    'Riak',
    'Ruby',
    'RunaWFE',
    'SAP',
    'SELinux',
    'SOLID',
    'SVM',
    'Samba',
    'Scala',
    'Scrum',
    'Serverless',
    'Sharepoint',
    'SkillsWiki',
    'Skype',
    'Software Defined Networks',
    'Strace',
    'Support',
    'TAU-платформа',
    'TDD',
    'Talks in English',
    'Tarantool',
    'Taucraft',
    'Team Communication',
    'Teмы',
    'ToPublish',
    'Tuukka Ahoniemi',
    'UI',
    'UI SmartTV',
    'UI бизнес-приложений',
    'UX',
    'UX + Agile',
    'UX проектирование',
    'User Story',
    'VCS',
    'Visual Studio',
    'Waterfall',
    'WebRTC',
    'Windows',
    'World Usability Day',
    'ZFS',
    'Zabbix',
    'Автоматизированное тестирование',
    'Алгоритмы',
    'Анализ программ и систем',
    'Аналитика',
    'Архитектура',
    'Архитектура информационных систем',
    'Архитектура серверных приложений',
    'Аутентификация и авторизация',
    'БПЛА',
    'Базы данных',
    'Байкал',
    'Безопасность',
    'Бизнес в IT',
    'Бизнес и СПО',
    'Бизнес-анализ',
    'Блиц-доклады',
    'Веб-дизайн',
    'Веб-разработка',
    'Вебинары PingWin',
    'Верификация',
    'Видеосвязь',
    'Визуализация',
    'Виртуализация',
    'Виртуальный ассистент',
    'Встраиваемые системы',
    'Выход на зарубежные рынки',
    'Геймификация',
    'Геймификация в UX',
    'Геолокация',
    'Голосовой интерфейс',
    'Госсектор',
    'Государство и софт',
    'Графовые базы данных',
    'Диаграммы Кано',
    'Дизайн',
    'Динамический анализ',
    'Дискуссии',
    'Дискуссии о юзабилити',
    'Доверенная загрузка',
    'Доклад со стенограммой',
    'Докладчики',
    'Доклады на английском',
    'Доклады на белорусском языке',
    'Доклады на иностранных языках',
    'Доклады на украинском',
    'Документация и Agile',
    'Документирование',
    'Запуск продукта',
    'Запуск продукта в Retail',
    'Инструменты майнтейнера',
    'Инструменты майнтейнеров',
    'Инструменты разработки',
    'Информационная безопасность',
    'Информационная безопасность и СПО',
    'Информационные системы ВУЗов',
    'Использование open-source',
    'Исследовательское тестирование',
    'История из практики',
    'Картография',
    'Карьера в IT',
    'Командообразование',
    'Компиляторы',
    'Компиляция под Linux',
    'Компьютерная графика',
    'Компьютерное зрение',
    'Конференции',
    'Корпоративные решения',
    'Криптография',
    'Кроссплатформенная разработка',
    'Круглый стол',
    'Кумир',
    'Лидерство',
    'Линейки конференций',
    'Локализация',
    'Маркетинг',
    'Мастер-классы',
    'Менеджмент',
    'Метрики качества',
    'Микропрограммирование',
    'Микросервисы',
    'Мобильная разработка',
    'Моделирование бизнес-процессов',
    'Моделирование физических систем',
    'Монетизация',
    'Мониторинг',
    'Мотивация',
    'Наука',
    'Облачные сервисы',
    'Образование',
    'Обучение',
    'Обучение бизнес-анализу',
    'Обучение бизнес-процессам',
    'Обучение проектному менеджменту',
    'Обучение системному программированию',
    'Онлайн-обучение',
    'Операционные системы',
    'Оптимизация приложения',
    'Опыт внедрения СПО',
    'Организационные изменения',
    'Организационный анализ',
    'Открытые данные',
    'Отладка',
    'Оценка сотрудников',
    'Очереди',
    'Параллельное программирование',
    'Пиктомир',
    'Планирование',
    'Планирование в Agile',
    'Планировка задач',
    'Привественные речи',
    'Программирование',
    'Программная архитектура',
    'Продуктовая аналитика',
    'Прототипирование UI',
    'Процесс разработки',
    'Процесс разработки UX и UI',
    'Процесс тестирования',
    'Психология пользователя',
    'Психология разработки',
    'Разработка open-source',
    'Разработка десктоп-приложений',
    'Разработка десктопных приложений под Windows',
    'Разработка игр',
    'Разработка операционных систем',
    'Распределенные системы',
    'Редкие языки программирования',
    'Реклама',
    'Реклама компании',
    'Рекомендательные системы',
    'Ретроспектива',
    'Рефакторинг',
    'Робототехника',
    'С++',
    'САПР',
    'СПО в Госуправлении',
    'СПО в России',
    'СПО в науке',
    'СПО в образовании',
    'СПО в творчестве',
    'СПО для системного администрирования',
    'СУБД',
    'Свободные библиотеки построения графиков',
    'Свободные лицензии',
    'Сервис-ориентированная архитектура',
    'Сети',
    'Системное администрирование',
    'Системное мышление',
    'Системный анализ',
    'Системный подход',
    'Системы управления версиями',
    'Скрытые категории',
    'Собрания ALT.NET',
    'Совещания',
    'Стажировка',
    'Статический анализ кода',
    'Стратегическое планирование',
    'ТРИЗ',
    'Темы',
    'Теория ограничений',
    'Тестирование',
    'Тестирование UI',
    'Тестирование игр',
    'Тестирование мобильных приложений',
    'Тестирование производительности',
    'Технологии',
    'Технологии будущего',
    'Технологии крупных вендоров',
    'Тренды Open-source',
    'Умные вещи',
    'Управление заинтересованными сторонами',
    'Управление качеством',
    'Управление продуктами',
    'Управление рисками',
    'Управление собой',
    'Управление техподдержкой',
    'Управление требованиями',
    'Файловые системы',
    'Философия программирования',
    'Философия юзабилити',
    'Финансовые системы',
    'Фреймворки',
    'Фриланс',
    'Функциональное программирование',
    'Хранение данных',
    'Хэши',
    'Эксплуатация',
    'Эльбрус',
    'Юзабилити',
    'Юзабилити в 1C',
    'Юзабилити в играх',
    'Юзабилити интернет-магазинов',
    'Юзабилити исследования',
    'Юзабилити мобильных устройств',
    'Юзабилити образование',
    'Юзабилити поиска',
    'Юзабилити текста',
    'Языки программирования',
])

In [8]:
def coalesce(dct, keys):
    for key in keys:
        if dct[key] is not None:
            return dct[key]
    return None


def parse_document_with_categories(doc):
    parsed_doc = dict()
    parsed_doc["title"] = doc["title"]
    parsed_doc['speakers'] = speaker_detect(doc['text'])
    
    sections_data = extract_sections(doc['text'])
    parsed_doc["annotation"] = sections_data["Аннотация"]
    parsed_doc["thesis"] = coalesce(
        sections_data, ["Thesis", "Тезисы", "Расширенные тезисы"]
    )
    
    parsed_doc['raw_categories'] = clear_categories(doc["categories"])
    parsed_doc["categories"] = list(
        filter(
            lambda x: x in ALL_CATEGORIES_WITHOUT_NAMES,
            parsed_doc['raw_categories'],
        )
    )
    return parsed_doc


data_raw = [parse_document_with_categories(doc) for doc in data['articles']]

Так как большие языковые модели довольно "умные", то можно спокойно опустить большую часть фильтрации текста.

Причём, если эту фильтрацию оставить, то полученные эмбеддинги окажутся не такими хорошими, поскольку обработанный текст менее логичен и связен, в отличии от естественного языка.

Тем не менее, мы оставим некоторые фильтры: это удаление ссылок и сохранение токена <sep> между разными модальностями

Очистка текста:
- удаление ссылок
- удаление всех HTML тегов

In [9]:
def clean_text_for_llm(text):
    if text is None or len(text) == 0:
        return ""

    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"http\S+|www\.\S+", "", text)

    return text


def clean_and_merge_document_for_llm(doc, sep_token=SEP_TOKEN):
    cleaned_texts = []

    # Clean each field and collect non-empty results
    for field in ["title", "annotation", "thesis"]:
        field_text = doc.get(field, "")
        cleaned_tokens = clean_text_for_llm(field_text)

        if cleaned_tokens:
            cleaned_texts.append(cleaned_tokens)

    # Join with <sep> token
    result_text = f" {sep_token} ".join(cleaned_texts).strip()

    result = {
        "text": result_text.split(),
        "categories": doc.get("categories", []),
        "raw_categories": doc.get("raw_categories", []),
        "speakers": doc.get("speakers", []),
    }

    return result


# Example usage
text = """
<html>Пример текста. Это тестовый текст с примером <a href="http://example.com">ссылки</a>.
Вот еще ссылка: https://www.test.ru.
Здесь есть часто-встречающиеся слова, например, "тест" много раз.
...
Также нужно удалить .эти строки.
"""

print(clean_text_for_llm(text))
print(clean_and_merge_document_for_llm(data_raw[1298]))


Пример текста. Это тестовый текст с примером ссылки.
Вот еще ссылка: 
Здесь есть часто-встречающиеся слова, например, "тест" много раз.
...
Также нужно удалить .эти строки.

{'text': ['Маркетинг', 'мобильных', 'приложений', '(Юрий', 'Мельничек,', 'SECR-2015)', '<sep>', 'Я', 'руковожу', 'MAPS.ME', 'с', 'самого', 'начала', 'и', 'до', '25', 'миллионов', 'инсталляций', 'на', 'текущий', 'момент.', 'За', 'это', 'время', 'у', 'меня', 'сформировался', 'системный', 'взгляд', 'на', 'маркетинг', 'мобильных', 'приложений,', 'которым', 'я', 'и', 'поделюсь', 'со', 'слушателями:', '*', 'Какие', 'существуют', 'маркетинговые', 'каналы', 'для', 'раскрутки', 'мобильных', 'приложений', '(PR,', 'ASO,', 'реклама,', 'фичеринги', 'и', 'т.д).', 'Как', 'работать', 'с', 'каждым', 'из', 'них.', '*', 'Какой', 'путь', 'проходят', 'пользователи', 'до', 'инсталляции', 'приложения.', '*', 'Как', 'увеличить', 'конверсию.', '*', 'Какие', 'есть', 'инструменты', 'мобильного', 'маркетинга.', '*', 'Какое', 'положение', 'ма

Process text

In [10]:
processed_data = [clean_and_merge_document_for_llm(doc) for doc in data_raw]

Process text with mBERT -- multi-lingual BERT

In [11]:
def encode_text(texts, model, tokenizer):
    encoding = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    device = model.device
    
    with torch.no_grad():
        outputs = model(
            input_ids=encoding["input_ids"].to(device),
            attention_mask=encoding["attention_mask"].to(device)
        )
    
    embeddings = outputs.last_hidden_state.mean(dim=(0, 1))
    return embeddings

In [12]:
IS_DOWNLOADED = True

if not IS_DOWNLOADED:
    model = AutoModel.from_pretrained("bert-base-multilingual-uncased")
    tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

    model.save_pretrained('models/m-bert')
    tokenizer.save_pretrained('tokenizers/m-bert')
else:
    model = AutoModel.from_pretrained("models/m-bert")
    tokenizer = AutoTokenizer.from_pretrained("tokenizers/m-bert")
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [13]:
processed_data_v2 = [
    {
        'text': encode_text(doc['text'], model, tokenizer).cpu().numpy(),
        'categories': doc['categories'],
        'raw_categories': doc['raw_categories'],
        'speakers': doc['speakers']
    } for doc in processed_data
]

Encode categories

In [14]:
label_embeddings = {ctgry: encode_text(ctgry, model, tokenizer).cpu().numpy() for ctgry in ALL_CATEGORIES_WITHOUT_NAMES} 

with open('data/label_embeddings.pkl', 'wb') as fd:
    pickle.dump(label_embeddings, fd)

Save embeddings into dataframe

In [15]:
df = pd.DataFrame(processed_data_v2)

unravelled_values = df['text'].explode().values.reshape(len(df), -1)
unravelled_df = pd.DataFrame(unravelled_values, dtype=float)
unravelled_df.columns = list(map(str, unravelled_df.columns))

df_final = pd.concat((df.iloc[:, 1:], unravelled_df), axis=1)
df_final.to_parquet('data/m_bert_data.parquet', index=False)